In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

2024-02-03 21:09:48.498899: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-03 21:09:48.501123: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-03 21:09:48.528920: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 21:09:48.528953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 21:09:48.529683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
threshold = 1

# Check if predicted value is threshold amount above or below actual value
def is_in_threshold(actual, pred):
    return pred <= actual + threshold and pred >= actual - threshold

def create_tf_column(results):
    return results.apply(lambda x: is_in_threshold(x['y_test'], x['predictions']), axis=1)

def get_score(y_test, y_pred, **kwargs):
    dataframe = pd.DataFrame(y_test, columns=['y_test'])
    dataframe['predictions'] = y_pred
    return create_tf_column(dataframe).value_counts().get(True) / dataframe.shape[0]

In [3]:
random_state=1

In [4]:
# Import data and preprocess
df = pd.read_csv("../test_data/Grounded CPW Leaky Wave antenna/S11 Data.csv")
df = df.drop(df[df['dB(S(1,1)) []'] > 0].index) # Remove all rows with positive s11

# # Split into x and y
# input_x = df.drop(columns=['dB(S(1,1)) []'], axis=1)
# input_y = df[['dB(S(1,1)) []']]

# Split data into training and testing
train_df = df.sample(frac=0.8, random_state=0)
test_df = df.drop(train_df.index)
X_train = train_df.copy()
X_test = test_df.copy()
y_train = X_train.pop('dB(S(1,1)) []')
y_test = X_test.pop('dB(S(1,1)) []')

In [5]:
# normalizer = tf.keras.layers.Normalization(axis=-1)
# normalizer.adapt(np.array(X_train))

In [6]:
import keras_tuner
class AntennaModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = keras.Sequential([
            layers.Dense(hp.Int('units_1', min_value=128, max_value= 160, step=32), activation='relu'),
            layers.Dense(1, activation='relu'),
        ])

        hp_learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default= 0.0005)

        model.compile(loss='mean_absolute_error',
                        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                        metrics=['mae','mse'])
        return model

    def fit(self, hp, model, x, y, **kwargs):
        if hp.Boolean("normalize"):
            x = tf.keras.layers.Normalization(axis=-1)(x)
        return model.fit(
            x,
            y,
            # Tune whether to shuffle the data in each epoch.
            shuffle=hp.Boolean("shuffle"),
            **kwargs,
        )

In [7]:
import keras_tuner
antenna_model = AntennaModel()
tuner = keras_tuner.RandomSearch(
    antenna_model.build,
    objective='val_loss',
    max_trials=20)

tuner.search_space_summary()
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Reloading Tuner from ./untitled_project/tuner0.json
Search space summary
Default search space size: 1
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 160, 'step': 32, 'sampling': 'linear'}


In [8]:
best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

In [9]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is {best_hps.get('units_1')}  

and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

# Evaluate the best model.
print(best_model.metrics_names)
loss, mae, mse = best_model.evaluate(X_test, y_test)
print(f'loss:{loss} mae: {mae} mse: {mse}')


The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is 128  

and the optimal learning rate for the optimizer
is 0.0005.

[]
27/27 [==============================] - 0s 1ms/step - loss: 10.0464 - mae: 10.0464 - mse: 150.3000
loss:10.046425819396973 mae: 10.046425819396973 mse: 150.29995727539062


In [22]:
tuned_model = tuner.hypermodel.build(best_hps)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min', restore_best_weights=False)
history_tuned= tuned_model.fit(X_train, y_train, 
          epochs = 200, 
          validation_data = (X_test, y_test),
          verbose=0,
          callbacks= early_stopping_cb)

print("\n Using Early stopping, needed only ",len(history_tuned.history['val_mse']),"epochs to converge!")

Epoch 21: early stopping

 Using Early stopping, needed only  21 epochs to converge!


In [13]:
X_test

,Feed_Gap [mm],Feed_Inset [cm],Feed_W [cm],Ground_gap [mm],Pad_L [cm],Pad_W [cm],Freq [GHz]
0,0.5,0.2,1.2,3,0.70,0.9,2.00
3,0.5,0.2,1.2,3,0.70,0.9,2.27
7,0.5,0.2,1.2,3,0.70,0.9,2.63
12,0.5,0.2,1.2,3,0.70,0.9,3.08
21,0.5,0.2,1.2,3,0.70,0.9,3.89
...,...,...,...,...,...,...,...
4194,0.5,0.2,1.2,4,0.75,1.2,17.66
4196,0.5,0.2,1.2,4,0.75,1.2,17.84
4202,0.5,0.2,1.2,4,0.75,1.2,18.38
4203,0.5,0.2,1.2,4,0.75,1.2,18.47


In [23]:
tuned_predicted = tuned_model.predict(X_test)
# get_score(y_test, tuned_predicted)

27/27 [==============================] - 0s 1ms/step


In [24]:
tuned_predicted

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [ ]:
%%time
history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=500)

NameError: name 'dnn_model' is not defined

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
test_results = dnn_model.evaluate(X_test, y_test, verbose=0)

In [ ]:
test_results

In [ ]:
test_predictions = dnn_model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')

plt.axline((-30,-30), (10,10))
plt.show()
# lims = [0, 50]
# plt.xlim(lims)
# plt.ylim(lims)
# _ = plt.plot(lims, lims)

In [ ]:
error = test_predictions - y_test
plt.hist(error, bins=25)
plt.xlabel('Prediction Error')
_ = plt.ylabel('Count')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler

model = DecisionTreeRegressor(random_state=random_state) # Initialize model
scaler = MinMaxScaler(feature_range=(0,1)) # Initialize scaler
pipeline = Pipeline(steps=[('normalize', scaler), ('model', model)]) # Create pipeline with scaler and model
pipeline_fit = pipeline.fit(X_train, y_train)
sklearn_predictions = pipeline_fit.predict(X_test)

In [ ]:
print(f"tensorflow score: {get_score(y_test.values.ravel(), test_predictions)}")
print(f"sklearn score: {get_score(y_test.values.ravel(), sklearn_predictions)}")

In [ ]:
analysis_df = X_test.copy()
analysis_df['dB(S(1,1)) []'] = y_test
analysis_df['tensorflow predicted s11'] = test_predictions
analysis_df['sklearn predicted s11'] = sklearn_predictions

In [ ]:
analysis_df

In [ ]:
matches

In [ ]:
what  = pd.concat([df, analysis_df])

In [ ]:
# what['tensorflow predicted s11'] = what['tensorflow predicted s11'].fillna(what['dB(S(1,1)) []'])
# what['sklearn predicted s11'] = what['sklearn predicted s11'].fillna(what['dB(S(1,1)) []'])

In [ ]:
import plotly.express as px

geometry = analysis_df.drop(['dB(S(1,1)) []', 'tensorflow predicted s11', 'sklearn predicted s11', 'Freq [GHz]'], axis=1).iloc[300]
geometry_filter = geometry.to_dict()

# Get all entries with all frequencies for this geometry 
matches = what.loc[(what[list(geometry_filter)] == pd.Series(geometry_filter)).all(axis=1)]

fig = px.line(matches.sort_values("Freq [GHz]"), x="Freq [GHz]", y=["dB(S(1,1)) []", 'tensorflow predicted s11', 'sklearn predicted s11'])

In [ ]:
fig.show()